In [1]:
#LIBRERIAS
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem.porter import *
import re
from unidecode import unidecode

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import LatentDirichletAllocation 
from scipy.sparse import csr_matrix 
import numpy as np

import pyLDAvis
from pyLDAvis import sklearn as sklearnlda

## LIMPIEZA INICIAL

In [28]:
df = pd.read_excel("2_final_db.xlsx", engine="openpyxl")

In [29]:
print(df["job_title"][0:10])

0                                           Comédienne
1                                   Empresa dedicada a
2    Actrice, chanteuse-auteure, Clown/Neztoile Ros...
3                                           Comédienne
4                          Senior Javascript Developer
5                                            Regisseur
6                                    Backend developer
7                                            Directora
8                                 Full Stack Developer
9                                 Senior IOS Developer
Name: job_title, dtype: object


In [30]:
#LIMPIEZA
df["job_title"]=df["job_title"].astype(str)
df["job_title"] = df["job_title"].str.lower() # minuscula
df["job_title"]=df["job_title"].str.replace('[>/,\.!?\-!?\n\)\(\r]', ' ') #eliminar caracteres raros
df["job_title"]=df["job_title"].str.replace(' +', ' ')
df["job_title"] = df["job_title"].map(lambda x: unidecode(x)) #homgeneizar

<>:4: DeprecationWarning: invalid escape sequence \.
<>:4: DeprecationWarning: invalid escape sequence \.
<ipython-input-30-fe7f9bf425bd>:4: DeprecationWarning: invalid escape sequence \.
  df["job_title"]=df["job_title"].str.replace('[>/,\.!?\-!?\n\)\(\r]', ' ') #eliminar caracteres raros


In [31]:
print(df["job_title"][0:10])

0                                           comedienne
1                                   empresa dedicada a
2    actrice chanteuse auteure clown neztoile rosa ...
3                                           comedienne
4                          senior javascript developer
5                                            regisseur
6                                    backend developer
7                                            directora
8                                 full stack developer
9                                 senior ios developer
Name: job_title, dtype: object


In [33]:
#FUNCIÓN PARA LEMATIZAR
# stemmer = PorterStemmer()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language="english")

def lem(texto):
    l = texto.split(" ")
    for e in range(len(l)):
        l[e] = stemmer.stem(l[e])
    text = " ".join(l)
    return text

df["job_title_l"] = df["job_title"].map(lambda x: lem(x))

In [34]:
#¿COMO SE VE CON LA LEMATIZACIÓN?
print(df["job_title_l"][0:10])

0                                            comedienn
1                                   empresa dedicada a
2    actric chanteus auteur clown neztoil rosa tapi...
3                                            comedienn
4                            senior javascript develop
5                                            regisseur
6                                      backend develop
7                                            directora
8                                   full stack develop
9                                    senior io develop
Name: job_title_l, dtype: object


## LDA

In [19]:
# Ahora construiremos la matriz término-documento
n_vocab=600
tf_vectorizer = CountVectorizer(max_df=0.8, min_df=2, max_features=n_vocab, stop_words='english', ngram_range=(1,3)) 
tf = tf_vectorizer.fit_transform(df["job_title"]) 
tf_feature_names = tf_vectorizer.get_feature_names()

In [20]:
# TF_detallada=pd.DataFrame(csr_matrix(tf).todense(), columns=tf_feature_names)
# TF_detallada.head()

In [21]:
# frecuencias=pd.DataFrame(TF_detallada.sum(), index=tf_feature_names, columns=['Freq'])
# frecuencias.sort_values(by=['Freq'], ascending=False, inplace=True)
# frecuencias.head(15)

In [22]:
#MODELO PARA TITLES SIN STEM

for i in range(3,11):
    lda = LatentDirichletAllocation(n_components=i, max_iter=11,doc_topic_prior=0.1, topic_word_prior=0.1, n_jobs=-1,random_state=353, verbose=1) #CONSTRUYO EL MODELO
    lda.fit(tf) # Esti
    LDAvis_prepared=sklearnlda.prepare(lda, tf, tf_vectorizer ) # Preparo el modelo y sus resultados para la visualización
    pyLDAvis.save_html(LDAvis_prepared, f'LDA_{i}.html')

iteration: 1 of max_iter: 11
iteration: 2 of max_iter: 11
iteration: 3 of max_iter: 11
iteration: 4 of max_iter: 11
iteration: 5 of max_iter: 11
iteration: 6 of max_iter: 11
iteration: 7 of max_iter: 11
iteration: 8 of max_iter: 11
iteration: 9 of max_iter: 11
iteration: 10 of max_iter: 11
iteration: 11 of max_iter: 11
iteration: 1 of max_iter: 11
iteration: 2 of max_iter: 11
iteration: 3 of max_iter: 11
iteration: 4 of max_iter: 11
iteration: 5 of max_iter: 11
iteration: 6 of max_iter: 11
iteration: 7 of max_iter: 11
iteration: 8 of max_iter: 11
iteration: 9 of max_iter: 11
iteration: 10 of max_iter: 11
iteration: 11 of max_iter: 11
iteration: 1 of max_iter: 11
iteration: 2 of max_iter: 11
iteration: 3 of max_iter: 11
iteration: 4 of max_iter: 11
iteration: 5 of max_iter: 11
iteration: 6 of max_iter: 11
iteration: 7 of max_iter: 11
iteration: 8 of max_iter: 11
iteration: 9 of max_iter: 11
iteration: 10 of max_iter: 11
iteration: 11 of max_iter: 11
iteration: 1 of max_iter: 11
iteratio

In [24]:
# Ahora construiremos la matriz término-documento
n_vocab=600
tf_vectorizer = CountVectorizer(max_df=0.8, min_df=2, max_features=n_vocab, stop_words='english', ngram_range=(1,3)) 
tf = tf_vectorizer.fit_transform(df["job_title_l"]) 
tf_feature_names = tf_vectorizer.get_feature_names()

In [27]:
#MODELO PARA TITLES CON STEM

for i in range(3,11):
    lda = LatentDirichletAllocation(n_components=i, max_iter=11,doc_topic_prior=0.1, topic_word_prior=0.1, n_jobs=-1,random_state=353, verbose=1) #CONSTRUYO EL MODELO
    lda.fit(tf) # Esti
    LDAvis_prepared=sklearnlda.prepare(lda, tf, tf_vectorizer ) # Preparo el modelo y sus resultados para la visualización
    pyLDAvis.save_html(LDAvis_prepared, f'L_LDA_{i}.html')

iteration: 1 of max_iter: 11
iteration: 2 of max_iter: 11
iteration: 3 of max_iter: 11
iteration: 4 of max_iter: 11
iteration: 5 of max_iter: 11
iteration: 6 of max_iter: 11
iteration: 7 of max_iter: 11
iteration: 8 of max_iter: 11
iteration: 9 of max_iter: 11
iteration: 10 of max_iter: 11
iteration: 11 of max_iter: 11
iteration: 1 of max_iter: 11
iteration: 2 of max_iter: 11
iteration: 3 of max_iter: 11
iteration: 4 of max_iter: 11
iteration: 5 of max_iter: 11
iteration: 6 of max_iter: 11
iteration: 7 of max_iter: 11
iteration: 8 of max_iter: 11
iteration: 9 of max_iter: 11
iteration: 10 of max_iter: 11
iteration: 11 of max_iter: 11
iteration: 1 of max_iter: 11
iteration: 2 of max_iter: 11
iteration: 3 of max_iter: 11
iteration: 4 of max_iter: 11
iteration: 5 of max_iter: 11
iteration: 6 of max_iter: 11
iteration: 7 of max_iter: 11
iteration: 8 of max_iter: 11
iteration: 9 of max_iter: 11
iteration: 10 of max_iter: 11
iteration: 11 of max_iter: 11
iteration: 1 of max_iter: 11
iteratio